In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install lightautoml

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd

# lightautoml
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.dataset.roles import NumericRole

import catboost
import lightgbm
import xgboost

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

from joblib import load, dump

In [ ]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS Jun"
path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - Jun 2021'
os.chdir(path)

In [ ]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
# target labelling
f = LabelEncoder()
f.fit(df_train['target'])
y = f.transform(df_train['target'])
y = pd.DataFrame(y).astype('int64')

col = [i for i in df_train.columns if i not in ['id', 'target']]
x = df_train[col]
x = x.astype('int64')

In [ ]:
PROJECT_NAME = 'lightautoml-20210608'
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 20210608
TEST_SIZE = 0.2
TIMEOUT = 8*60*60   # in seconds
TARGET_NAME = 'target'

In [ ]:
# create task
task = Task('multiclass', loss = 'crossentropy', metric = 'crossentropy')

# setup columns
roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

In [ ]:
# train on full data


automl = TabularAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {
                           'use_algos': [['linear_l2', 'lgb', 'lgb_tuned', 'cb', 'cb_tuned']],
                           'return_all_predictions': True,
                           'weighted_blender_max_nonzero_coef': 0.0
                       },
                       tuning_params = {'max_tuning_time': 60*60},
                       reader_params = {'n_jobs': N_THREADS, 'random_state': RANDOM_STATE},
                       config_path = f'lightautoml-configs\{PROJECT_NAME}.yml'
                       )

oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

In [ ]:
# predict for test data
test_pred = automl.predict(test_data)

first_pred = pd.DataFrame(test_pred)
print(first_pred.head())
print(first_pred.shape)
first_pred.to_csv(f'{PROJECT_NAME}_1st_pred.csv')

# print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

# print('Check scores...')
# print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

In [ ]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result
sub.to_csv(f'{RESULTS_PATH}.csv', index=False)